In [1]:
import os 

from dotenv import load_dotenv
load_dotenv()


OLLAMA_URL=os.getenv("OLLAMA_URL")
LLM_MODEL=os.getenv("LLM_MODEL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
DATAPATH=os.getenv("DATAPATH")
#OLLAMA_URL='http://localhost:11434'
#LLM_MODEL="llama3.2:3b" # max retires
#LLM_MODEL="gemma3:4b" # no tool support
#LLM_MODEL="llava:7b"
#LLM_MODEL="codellama:7b"
#LLM_MODEL="qwen2.5-coder:1.5b" # no tool support
#LLM_MODEL="deepseek-r1:1.5b" # does not support tools
#os.setenv("no_proxy=localhost,127.0.0.1,*.my-it-solutions.net,ollama")

print(OLLAMA_URL)
print(f"model: {LLM_MODEL}")
print(f"embedding: {EMBEDDING_MODEL}")
print(f"datapath {DATAPATH}")
print(f"no_proxy {os.getenv('no_proxy')}")

import requests
import time

response = requests.get(OLLAMA_URL,timeout=3)
if response.status_code == requests.codes.ok:
    print(response)
else:
    print(response)    

http://ollama-service-internal.default.svc.cluster.local:11433
model: llama3.2:3b
embedding: bge-m3:567m
datapath /app/data
no_proxy None
<Response [200]>


In [2]:
#from llama_index.llms.ollama import Ollama
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
#llm = Ollama(model=LLM_MODEL, request_timeout=120.0, base_url=OLLAMA_URL, temperature=0.0)


ollama_model = OpenAIModel(
    model_name='LLM_MODEL', provider=OpenAIProvider(base_url=f"{OLLAMA_URL}/v1")
)


In [12]:
from mcp.server.fastmcp import FastMCP

from starlette.applications import Starlette
from starlette.routing import Mount, Host

from pydantic_ai import Agent
import asyncio
import nest_asyncio
nest_asyncio.apply()
#loop = asyncio.get_event_loop()


server = FastMCP('PydanticAI Server')
server_agent = Agent(model=ollama_model,
              result_type=str,
              deps_type=str,
              system_prompt="",              
              retries=2)

@server.tool()
async def poet(theme: str) -> str:
    """Poem generator"""
    r = await server_agent.run(f'write a poem about {theme}')
    return r.data

@server.prompt()
def review_code(code: str) -> str:
    return f"Please review this code:\n\n{code}"    


#server.run()
#asyncio.run_coroutine_threadsafe(server.run('sse'), loop)
#asyncio.run(server.run('sse'))
# Mount the SSE server to the existing ASGI server
app = Starlette(
    routes=[
        Mount('/', app=server.sse_app()),
    ]
)

# or dynamically mount as host
app.router.routes.append(Host('localhost', app=server.sse_app()))


ModuleNotFoundError: No module named 'fastmcp'

In [15]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerHTTP

server = MCPServerHTTP(url='http://localhost:8888/sse')  
agent = Agent(model=ollama_model,
              result_type=str,
              deps_type=str,
              system_prompt="",
              mcp_servers=[server],
              retries=1)


async def main():
    async with agent.run_mcp_servers():  
        result = await agent.run('How many days between 2000-01-01 and 2025-03-18?')
    print(result.data)
    #> There are 9,208 days between January 1, 2000, and March 18, 2025.

result = await main()
print(result.data)

[03/27/25 23:06:25] INFO     Connecting to SSE endpoint: http://localhost:8888/sse                        ]8;id=718312;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=871641;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py#45\45]8;;\

                    INFO     HTTP Request: GET http://localhost:8888/sse "HTTP/1.1 404 Not Found"   ]8;id=796581;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=998389;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\

  + Exception Group Traceback (most recent call last):
  |   File "/opt/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_858/880284894.py", line 19, in <module>
  |     result = await main()
  |              ^^^^^^^^^^^^
  |   File "/tmp/ipykernel_858/880284894.py", line 14, in main
  |     async with agent.run_mcp_servers():
  |                ~~~~~~~~~~~~~~~~~~~~~^^
  |   File "/usr/local/lib/python3.13/contextlib.py", line 214, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "/opt/venv/lib/python3.13/site-packages/pydantic_ai/agent.py", line 1299, in run_mcp_servers
  |     await exit_stack.enter_async_context(mcp_server)
  |   File "/usr/local/lib/python3.13/contextlib.py", line 668, in enter_async_context
  |     result = await _enter(cm)
  |              ^^^^^^^^^^^^^^^^
  |   File "/opt/venv/lib/p